<a href="https://colab.research.google.com/github/lblogan14/reinforcement_learning_with_tensorflow/blob/master/ch6_asynchronous.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Review:
![alt text](https://github.com/lblogan14/reinforcement_learning_with_tensorflow/blob/master/note_images/ch4/actor-critic.PNG?raw=true)

Recall the structure of the actor-critic algorithm from Chapter 4. \\
The **Actor** takes the current environment state and determines best action to take; \\
the **Critic** plays a policy-evaluation role by taking in the environment state and action, and then returns a score depicting how good an action is for the state.

Thus, the actor-critic algorithm learns both the policy and state-action value function.

#Asynchronous Methods
Deep Q-network utilizes the experience replay to train the deep neural network in order to find out the maximum Q-value for the most favorable action, but it takes too much memory usage and heavy computation over time. Thus, the asynchronous method is to overcome this issue. Instead of using experience replay, in asynchronous methods, multiple instances of the environment are created and multiple agents asynchronously execute actions in parallel:
![alt text](https://github.com/lblogan14/reinforcement_learning_with_tensorflow/blob/master/note_images/ch6/asy_method.PNG?raw=true)

Thus, each thread is assigned the process that contains a learner representing an agent network that interacts with its own copy of the environment. Multiple learners run in parallel exploring their own environment. The parallelism allows the agent to experience varied different states simultaneously at any given time-step, and covers the fundamentals of both off-policy and on-policy learning algorithms. These multiple learners running in parallel use different exploration policies, which maximizes the diversity. Different exploration policies by different learners changes the parameters, and these updates have the least chance to be correlated in time. Therefore, experience replay memory is not required.

Examples of asynchronous methods:
* Asynchronous one-step Q-learning
* Asynchronous one-step SARSA
* Asynchronous n-step Q-learning
* Asynchronous advantage actor critic (A3C)

#Asynchronous one-step Q-learning
An agent in DQN is represented by a set of primary and target networks, where **one-step loss** is calculated as the square of the difference between the state-action value of the current state s predicted by the primary network and the target state-action value of the current state calculated by the target network. (Similar to DQN)

(New for Asynchronous) There are multiple learning agents running and calculating the one-step loss in parallel. Thus the gradient calculation occurs in parallel in different threads where each learning agent interacts with its own copy of environment. The accumulation of these gradients in
different threads over multiple time steps are used to update the policy network parameters
after a fixed time step, or when an episode is over. The accumulation of gradients is
preferred over policy network parameter updates because this avoids overwriting the
changes perform by each of the learner agents.

Adding a different exploration policy to different threads makes the learning
diverse and robust. This improves the performance owing to better exploration, because
each of the learning agents in different threads is subjected to a different exploration policy.

Pseudocode for Asynchronous one-step Q-learning: \\
where \\
* $\theta$: parameters of the policy network
* $\theta^t$: parameters of the target network
* $T$: overal time step counter

`// Globally shared parameters ` $\theta, \theta^t$ and $T$ \\
`// ` $\theta$ `is initialized arbitrarily` \\
`// T is initialized 0` \\
 \\
`Initialize thread level time step counter t=0` \\
`Initialize ` $\theta^t=\theta$ \\
`Initialize network gradients` $d\theta=0$ \\
`Start with the initial state s` \\
`repeat until ` $T>T_{\max}:$ \\
$\quad$ `Choose action a with ` $\epsilon$ `-greedy policy such that:` \\
$\quad\quad a=\{\begin{array}{l}a\,random\, action\quad ,with\,probability\,\epsilon \\ \arg\max_{a'}Q(\phi(s),a';\theta) \quad otherwise \end{array}$ \\
$\quad$ `Perform action a` \\
$\quad$ `Receive new state s' and reward r` \\
$\quad$ `Compute target y: ` $y = \{ \begin{array}{l}
                                                 r \quad , for\,terminal\,s'\\ 
                                                 r+\gamma\max_{a'}Q(s',a';\theta_t) \quad ,otherwise 
                                                 \end{array}$ \\
$\quad$ `Compute the loss, ` $L(\theta)=(y-Q(s,a;\theta))^2$ \\
$\quad$ `Accumulate the gradient w.r.t. ` $\theta : d\theta=d\theta+\frac{\nabla L(\theta)}{\nabla\theta}$ \\
$\quad$ `s = s'` \\
$\quad$ `T = T + 1` \\
$\quad$ `t = t + 1` \\
 \\
$\quad$ `if T mod ` $I_{target}==0:$ \\
$\quad\quad$ `Update the parameters of target network: ` $\theta^t=\theta$ \\
$\quad\quad$ `# After every ` $I_{target}$ `time steps the parameters of target network is updated` \\
 \\
$\quad$ `if t mod ` $I_{AsyncUpdate}==0$ `or s = terminal state:` \\
$\quad\quad$ `Asynchronous update of ` $\theta$ `using ` $d\theta$ \\
$\quad\quad$ `Clear gradients: ` $d\theta=0$ \\
$\quad\quad$ `#at every ` $I_{AsyncUpdate}$ `time step in the thread or if s is the terminal state` \\
$\quad\quad$ `# update ` $\theta$ `using accumulated gradients ` $d\theta$

#Asynchronous one-step SARSA
uses $\epsilon$-greedy to choose the action $a'$ for the next state $s'$ and the Q-value of the next state-action pair: $Q(s',a'; \theta^t)$ is used to calculate the target state-action value of the current state.

Pseudocode for Asynchronous one-step SARASA: \\
where \\
* $\theta$: parameters of the policy network
* $\theta^t$: parameters of the target network
* $T$: overal; time step counter

`// Globally shared parameters ` $\theta, \theta^t$ and $T$ \\
`// ` $\theta$ `is initialized arbitrarily` \\
`// T is initialized 0` \\
 \\
`Initialize thread level time step counter t=0` \\
`Initialize ` $\theta^t=\theta$ \\
`Initialize network gradients` $d\theta=0$ \\
`Start with the initial state s` \\
`Choose action a with ` $\epsilon$ `-greedy policy such that:` \\
$\quad a=\{\begin{array}{l}a\,random\, action\quad ,with\,probability\,\epsilon \\ \arg\max_{a'}Q(\phi(s),a';\theta) \quad otherwise \end{array}$ \\
`repeat until ` $T>T_{\max}$ : \\
$\quad$ `Perform action a` \\
$\quad$ `Receive new state s' and reward r` \\
$\quad$ `Choose action a' with ` $\epsilon$ `-greedy policy such that:` \\
$\quad\quad a'=\{\begin{array}{l}a\,random\, action\quad ,with\,probability\,\epsilon \\ \arg\max_{a'}Q(\phi(s),a'';\theta) \quad otherwise \end{array}$ \\
$\quad$ `Compute target y: ` $y = \{ \begin{array}{l}
                                                 r \quad , for\,terminal\,s'\\ 
                                                 r+\gamma\max_{a'}Q(s',a';\theta_t) \quad ,otherwise 
                                                 \end{array}$ \\
$\quad$ `Compute the loss, ` $L(\theta)=(y-Q(s,a;\theta))^2$ \\
$\quad$ `Accumulate the gradient w.r.t. ` $\theta : d\theta=d\theta+\frac{\nabla L(\theta)}{\nabla\theta}$ \\
$\quad$ `s = s'` \\
$\quad$ `T = T + 1` \\
$\quad$ `t = t + 1` \\
$\quad$ `a = a'` \\
 \\
$\quad$ `if T mod ` $I_{target}==0:$ \\
$\quad\quad$ `Update the parameters of target network: ` $\theta^t=\theta$ \\
$\quad\quad$ `# After every ` $I_{target}$ `time steps the parameters of target network is updated` \\
 \\
$\quad$ `if t mod ` $I_{AsyncUpdate}==0$ `or s = terminal state:` \\
$\quad\quad$ `Asynchronous update of ` $\theta$ `using ` $d\theta$ \\
$\quad\quad$ `Clear gradients: ` $d\theta=0$ \\
$\quad\quad$ `#at every ` $I_{AsyncUpdate}$ `time step in the thread or if s is the terminal state` \\
$\quad\quad$ `# update ` $\theta$ `using accumulated gradients ` $d\theta$

#Asynchronous n-step Q-learning
similar to asynchronous one-step Q-learning, but for asynchronous n-step Q-learning, the learning agent actions are selected using the exploration policy for up to $t_{\max}$ steps or until a terminal state is reached, in order to compute a single update of policy network parameters. The loss for each time step is calculated as the difference between the discounted future rewards at that time step and the estimated Q-value.

The loss gradient with respect to thread-specific network parameters for each time step is calculated and accumulated. There are multiple such
learning agents running and accumulating the gradients in parallel. These accumulated
gradients are used to perform asynchronous updates of policy network parameters.

Pseudocode for asynchronous n-step Q-learning: \\
where \\
* $\theta$: parameters of the policy network
* $\theta^t$: parameters of the target network
* $T$: overall time step counter
* $t$: thread level time step counter
* $T_{\max}$: maximum number of overall time steps
* $t_{\max}$: maximum number of time steps in a thread

`// Globally shared parameters ` $\theta, \theta^t$ and $T$ \\
`// ` $\theta$ `is initialized arbitrarily` \\
`// T is initialized 0` \\
 \\
`Initialize thread level time step counter t=0` \\
`Initialize ` $\theta^t=\theta$ \\
`Initialize ` $\theta'=\theta$ \\
`Initialize network gradients` $d\theta=0$ \\
 \\
`repeat until ` $T>T_{\max}$: \\
$\quad$ `Clear gradient: ` $d\theta=0$ \\
$\quad$ `Synchronize thread-specific parameters: ` $\theta'=\theta$ \\
$\quad t_{start}=t$ \\
$\quad$ `Get state ` $s_t$ \\
$\quad$ `r = [] // list of rewards` \\
$\quad$  `a = [] // list of actions` \\
$\quad$ `s = [] // list of states` \\
$\quad$ `repeat until ` $s_t$ `is a terminal state or ` $t-t_{start}==t_{\max}$: \\
$\quad\quad$ `Choose action` $a_t$  `with ` $\epsilon$ `-greedy policy such that:` \\
$\quad\quad\quad a_t=\{\begin{array}{l}a\,random\, action\quad ,with\,probability\,\epsilon \\ \arg\max_{a_t}Q(\phi(s),a'';\theta) \quad otherwise \end{array}$ \\
$\quad\quad$ `Perform action` $a_t$ \\
$\quad\quad$ `Receive new state ` $s_{t+1}$ `and reward ` $r_t$ \\
$\quad\quad$ `Accumulate rewards by appending ` $r_t$ `to r` \\
$\quad\quad$ `Accumulate actions by appending ` $a_t$ `to a` \\
$\quad\quad$ `Accumulate states by appending ` $s_t$ `to s` \\
$\quad\quad$ `t = t + 1` \\
$\quad\quad$ `T = T + 1` \\
$\quad\quad s_t=s_{t+1}$ \\
$\quad$ `Compute returns, R:` $R=\{\begin{array}{l}0\quad ,\,for\,terminal\,s_t \\ \max_{a}Q(s_t,a;\theta^t) \quad otherwise \end{array}$ \\
$\quad$ `for ` $i\in[t-1,......, t_{start}]$ `do:` \\
$\quad\quad R=r_i+\gamma R$ \\
$\quad\quad$ `Compute loss, ` $L(\theta')=(R-Q(s_i,a_i;\theta'))^2$ \\
$\quad\quad$ `Accumulate gradients w.r.t. ` $\theta': d\theta=d\theta+\frac{\nabla L(\theta')}{\nabla\theta'}$ \\
$\quad$ `Asynchronous update of ` $\theta$ `using ` $d\theta$ \\
$\quad$ `if T mod ` $I_{target}==0:$ \\
$\quad\quad$ `Update the parameters of target network: ` $\theta^t=\theta$ \\
$\quad\quad$ `# After every ` $I_{target}$ `time steps the parameters of target network is updated` \\

#Asynchronous Advantage Actor Critic (A3C)
Each learning agent contains an actor-critic learner that combines the benefits of both value-based and policy-based methods. \\
The
actor network takes in the state as input and predicts the best action of that state, while the
critic network takes in the state and action as the inputs and outputs the action score to
quantify how good the action is for that state. The actor network updates its weight
parameters using policy gradients, while the critic network updates its weight parameters using *TD(0)* method, the difference of value estimates between two time steps. \\
Recall that the **advantage function** is the difference between the expected future rewards and the baseline function. It tells the good or bad status and how good or bad that action was expected to be.

#A3C for `Pong-v0` in OpenAI Gym

In [0]:
import multiprocessing
import threading
import tensorflow as tf
import numpy as np
import gym
import os
import shutil
import matplotlib.pyplot as plt

The input state image preprocessing function is required:

In [0]:
def preprocessing_image(obs): #where I is the single frame of game as input
  '''preprocessing 210x160x3 frame into 6400 (80x80) 1D float vector'''
  #the values below have been precomputed through trial and error by OpenAI team
  obs = obs[35:195] #crop image where it contains only the paddles
  obs = obs[::2,::2,0] #downsample by factor of 2 and take only R channel
  obs[obs==144] = 0 #erase background type 1
  obs[obs==109] = 0 #erase background type 2
  obs[obs!=0] = 1 #everything else set to 1
  return obs.astype('float').ravel() #flatten to 1D

Set up the environment:

In [0]:
game_env = 'Pong-v0'
num_workers = multiprocessing.cpu_count()
max_global_episodes = 100000
global_network_scope = 'globalnet'
global_iteration_update = 20
gamma = 0.9
beta = 0.0001
lr_actor = 0.0001 # learning rate for actor
lr_critic = 0.0001 # learning rate for critic
global_running_rate = []
global_episode = 0

env = gym.make(game_env)

num_actions = env.action_space.n

tf.reset_default_graph()

Define the `actor-critic` class which contains the architecture of `actor` and `critic` network:

In [0]:
class ActorCriticNetwork(object):
  def __init__(self, scope, globalAC=None):
    if scope == global_network_scope: # get global net
      with tf.variable_scope(scope):
        self.s = tf.placeholder(tf.float32, [None, 6400], 'state')
        self.a_params, self.c_params = self._build_net(scope)[-2:]
    else: # get local net, calculate losses
      with tf.variable_scope(scope):
        self.s = tf.placeholder(tf.float32, [None, 6400], 'state')
        self.a_his = tf.placeholder(tf.int32, [None,], 'action')
        self.v_target = tf.placeholder(tf.float32, [None, 1], 'target_vector')
        
        self.a_prob, self.v, self.a_params, self.c_params = self._build_net(scope)
        
        td = tf.subtract(self.v_target, self.v, name='temporal_difference_error')
        with tf.name_scope('critic_loss'):
          self.c_loss = tf.reduce_mean(tf.square(td))
          
        with tf.name_scope('actor_loss'):
          log_prob = tf.reduce_sum(tf.log(self.a_prob) * tf.one_hot(self.a_his, num_actions, dtype=tf.float32), axis=1, keepdims=True),
          exp_v = log_prob * td
          entropy = -tf.reduce_sum(self.a_prob * tf.log(self.a_prob + 1e-5), axis=1, keepdims=True) #exploration
          self.exp_v = beta * entropy + exp_v
          self.a_loss = tf.reduce_mean(-self.exp_v)
          
        with tf.name_scope('local_grad'):
          self.a_grads = tf.gradients(self.a_loss, self.a_params)
          self.c_grads = tf.gradients(self.c_loss, self.c_params)
          
      with tf.name_scope('sync'):
        with tf.name_scope('pull'):
          self.pull_a_params_op = [l_p.assign(g_p) for l_p, g_p in zip(self.a_params, globalAC.a_params)]
          self.pull_c_params_op = [l_p.assign(g_p) for l_p, g_p in zip(self.c_params, globalAC.c_params)]
          
        with tf.name_scope('push'):
          self.update_a_op = actor_train.apply_gradients(zip(self.a_grads, globalAC.a_params))
          self.update_c_op = critic_train.apply_gradients(zip(self.c_grads, globalAC.c_params))
          
          
  def _build_net(self, scope):
    w_init = tf.random_normal_initializer(0., .1)
    
    with tf.variable_scope('actor_network'):
      l_a = tf.layers.dense(self.s, 
                            300, 
                            tf.nn.relu6, 
                            kernel_initializer=w_init, 
                            name='actor_layer')
      a_prob = tf.layers.dense(l_a, 
                               num_actions, 
                               tf.nn.softmax,
                               kernel_initializer=w_init,
                               name='ap')
      
    with tf.variable_scope('critic_network'):
      l_c = tf.layers.dense(self.s,
                            100,
                            tf.nn.relu6,
                            kernel_initializer=w_init,
                            name='critic_layer')
      v = tf.layers.dense(l_c, 
                          1, 
                          kernel_initializer=w_init,
                          name='v') # state value
      
      a_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                                   scope=scope + '/actor')
      c_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                                   scope=scope + '/critic')
      return a_prob, v, a_params, c_params
    
    
  def update_global(self, feed_dict): # run local
    session.run([self.update_a_op, self.update_c_op], feed_dict)
    # local gradient applied to global net
    
    
  def pull_global(self): # run local
    session.run([self.pull_a_params_op, self.pull_c_params_op])
    
    
  def choose_action(self, s): # run local
    s = np.reshape(s, [-1])
    prob_weights = session.run(self.a_prob,
                               feed_dict={self.s: s[np.newaxis, :]})
    action = np.random.choice(range(prob_weights.shape[1]),
                              p=prob_weights.ravel()) # select action w.r.t the actions prob
    return action

Define the `worker` class which represents the process in each thread:

In [0]:
class Worker(object):
  def __init__(self, name, globalAC):
    self.env = gym.make(game_env).unwrapped
    self.name = name
    self.AC = ActorCriticNetwork(name, globalAC)
    
    
  def work(self):
    global global_running_rate, global_episode
    total_step = 1
    buffer_s, buffer_a, buffer_r = [], [], []
    
    while not coordinator.should_stop() and global_episode < max_global_episodes:
      obs = self.env.reset()
      s = preprocessing_image(obs)
      ep_r = 0
      
      while True:
        if self.name == 'W_0':
          self.env.render(mode = 'rgb_array')
        a = self.AC.choose_action(s)
        
        print(a.shape)
        
        obs_, r, done, info = self.env.step(a)
        s_ = preprocessing_image(obs_)
        if done and r<=0:
          r = -20
          
        ep_r += r
        buffer_s.append(np.reshape(s, [-1]))
        buffer_a.append(a)
        buffer_r.append(r)
        
        if total_step % global_iteration_update == 0 or done: 
          # update global and assign to local net
          if done:
            v_s = 0 # terminal
          else:
            s_ = np.reshape(s_, [-1])
            v_s_ = session.run(self.AC.v, {self.AC.s: s_[np.newaxis, :]})[0, 0]
            
          buffer_v_target = []
          for r in buffer_r[::-1]: #reverse buffer r
            v_s_ = r + gamma * v_s_
            buffer_v_target.append(v_s_)
          buffer_v_target.reverse()
          
          buffer_s, buffer_a, buffer_v_target = np.vstack(buffer_s), np.array(buffer_a), np.vstack(buffer_v_target)
          feed_dict = {self.AC.s: buffer_s,
                       self.AC.a_his: buffer_a,
                       self.AC.v_target: buffer_v_target}
          self.AC.update_global(feed_dict)
          
          buffer_s, buffer_a, buffer_r = [], [], []
          self.AC.pull_global()
          
        s = s_
        total_step += 1
        
        if done:
          if len(global_running_rate) == 0: 
            # record running episode reward
            global_running_rate.append(ep_r)
          else:
            global_running_rate.append(0.99 * global_running_rate[-1] + 0.01 * ep_r)
          print(self.name,
                'Ep:', global_episode,
               '| Ep_r: %i' % global_running_rate[-1])
          global_episode += 1
          break

Now we can create the thread pool and assign workers to different threads:

In [0]:
session = tf.Session()

with tf.device('/cpu:0'):
  actor_train = tf.train.RMSPropOptimizer(lr_actor, name='RMSPropOptimizerActor')
  critic_train = tf.train.RMSPropOptimizer(lr_critic, name='RMSPropOptimizerCritic')
  acn_global = ActorCriticNetwork(global_network_scope)
  workers = []
  
  #create workers
  for i in range(num_workers):
    i_name = 'W_%i' % i # worker name
    workers.append(Worker(i_name, acn_global))
    
coordinator = tf.train.Coordinator()
session.run(tf.global_variables_initializer())

worker_threads = []
for worker in workers:
  job = lambda: worker.work()
  t = threading.Thread(target=job)
  t.start()
  worker_threads.append(t)
coordinator.join(worker_threads)

plt.plot(np.arange(len(global_running_rate)), global_running_rate)
plt.xlabel('step')
plt.ylabel('Total moving reward')
plt.show()